# Hospital Length of Stay

In order for hospitals to optimize resource allocation, it is important to predict accurately how long a newly admitted patient will stay in the hospital.

This notebook takes advantage of the power of SQL Server and RevoScalePy. The tables are all stored in a SQL Server, and most of the computations are done by loading chunks of data in-memory instead of the whole dataset.

It does the following: 

 * **Step 0: Packages and Compute Contexts**
 * **Step 1: Processing and Cleaning**
 * **Step 2: Feature Engineering**
 * **Step 3: Training and Evalutating a Random Forest, Boosted Trees, Fast Trees, and a Neural Network**

## Step 0: Packages and Compute Contexts

#### In this step, we set up the connection string to access a SQL Server Database and load the necessary packages. 

In [1]:
# WARNING.
# We recommend not using Internet Explorer as it does not support plotting, and may crash your session.

In [2]:
# Load packages.
import os, sys
from numpy import mean
from math import sqrt
from pandas import Series, DataFrame, to_numeric
import pyodbc

from revoscalepy import RxInSqlServer, RxLocalSeq, rx_set_compute_context, rx_import, rx_data_step, rx_summary
from revoscalepy import rx_get_var_names, RxSqlServerData, RxTextData, rx_serialize_model, rx_import, rx_data_step
from revoscalepy import rx_set_compute_context, rx_import, rx_data_step, rx_summary
from revoscalepy import rx_dforest, rx_btrees, rx_predict, RxOdbcData, rx_get_var_info, RxSqlServerData, rx_get_var_names
from revoscalepy import RxInSqlServer, RxLocalSeq, rx_set_compute_context, rx_write_object

from microsoftml import rx_fast_trees, rx_neural_network, adadelta_optimizer
from microsoftml import rx_predict as ml_predict

from length_of_stay_utils import train_test_split, create_formula, write_rts_model, evaluate_model, get_num_rows, drop_view, alter_column

In [3]:
# Define Compute Contexts: user to input Server Name, database name, UID and Password. 
from SQLConnection import *

# Choose a database name and create it. 
db = "Hospital"

## Create database. 
pyodbc_cnxn = pyodbc.connect(connection_string)
pyodbc_cursor = pyodbc_cnxn.cursor()
pyodbc_cursor.execute("if not exists(SELECT * FROM sys.databases WHERE name = '{}') CREATE DATABASE {};".format(db, db))
pyodbc_cursor.close()
pyodbc_cnxn.commit()
pyodbc_cnxn.close()

## Step 1: Pre-Processing and Cleaning

In this step, we: 

**1.** Upload the data set to SQL.

**2.** Clean the merged data set: we replace NAs with the mode (categorical variables) or mean (continuous variables).

**Input:**  Data Set LengthOfStay.csv

**Output:** Cleaned raw data set LoS.

In [4]:
# Set the compute context to Local. 
rx_set_compute_context(local)

In [5]:
# Upload the data set to SQL.
## Point to the input data set while specifying the classes.
file_path = "..\\Data"
LoS_text = RxTextData(file = os.path.join(file_path, "LengthOfStay.csv"), column_info=col_type_info)

## Upload the table to SQL. 
LengthOfStay_sql = RxSqlServerData(table = "LengthOfStay", connection_string = connection_string)
rx_data_step(input_data = LoS_text, output_file = LengthOfStay_sql, overwrite = True)

print("Data exported to SQL")

Rows Read: 100000, Total Rows Processed: 100000, Total Chunk Time: 4.272 seconds 

Elapsed time to compute low/high values and/or factor levels: 4.477 secs.
 
Total Rows written: 100000, Total time: 5.052
Rows Read: 100000, Total Rows Processed: 100000, Total Chunk Time: 8.087 seconds 
Data exported to SQL


In [6]:
# Determine if LengthOfStay has missing values

table = "LengthOfStay"

# First, get the names and types of the variables to be treated.
data_sql = RxSqlServerData(table = table, connection_string = connection_string, stringsAsFactors = True)
colnames = rx_get_var_names(data_sql)

# Then, get the names of the variables that actually have missing values. Assumption: no NA in eid, lengthofstay, or dates. 
var = [x for x in colnames if x not in ["eid", "lengthofstay", "vdate", "discharged"]]
f = "+".join(var)
summary = rx_summary(formula = f, data = data_sql, by_term = True).summary_data_frame

var_with_NA = summary[summary["MissingObs"] > 0]

method = None
if var_with_NA.empty:
    print("No missing values.")
    print("You can move to step 2.")
    missing = False
else:
    print("Variables containing missing values are:")
    print(var_with_NA)
    print("Apply one of the methods below to fill missing values.")
    missing = True

Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 2.337 seconds
Rows Read: 50000, Total Rows Processed: 100000, Total Chunk Time: 2.653 seconds 
Computation time: 5.151 seconds.
No missing values.
You can move to step 2.


In [7]:
# If applicable, NULL is replaced with the mode (categorical variables: integer or character) or mean (continuous variables).

if(missing == False):
    print("Nothing to clean")
    LengthOfStay_cleaned_sql = RxSqlServerData(table = table, connection_string = connection_string)
else:
    print("Fill with mode and mean")

    # Get the variables types (categortical vs. continuous)
    categ_names = []
    contin_names = []
    for index, row in var_with_NA.iterrows():
        nameSeries = var_with_NA["Name"]
        name = nameSeries.to_string().split()[-1]
        if col_info[name]["type"] == "numeric":
            contin_names.append(name)
        else:
            categ_names.append(name)

    # Function to replace missing values with the mode (categorical variables) or mean (continuous variables)
    def fill_NA_mode_mean(dataset, context):
        data = DataFrame(dataset)
        for name in categ_names:
            data.loc[data[name].isnull(),name] = data[name].mode().iloc[0]
        for name in contin_names:
            data.loc[data[name].isnull(), name] = data[name].mean()
        return data

    # Apply this function to LengthOfStay by wrapping it up in rxDataStep. Output is written to LoS0.
    # We drop the LoS0 view in case the SQL Stored Procedure was executed in the same database before.
    pyodbc_cnxn = pyodbc.connect(connection_string)
    drop_view("LoS0", connection_string)

    LoS0_sql = RxSqlServerData(table = "LoS0", connection_string = connection_string)
    rx_data_step(input_data = LengthOfStay_sql, output_file = LoS0_sql, overwrite = True, transform_function = fill_NA_mode_mean)
   
    LengthOfStay_cleaned_sql = RxSqlServerData(table = "LoS0", connectionString = connection_string)    
    
print("Data cleaned")

Nothing to clean
Data cleaned


## Step 2: Feature Engineering

In this step, we:

**1.** Standardize the continuous variables (Z-score).

**2.** Create the variable number_of_issues: the number of preidentified medical conditions.

**Input:** Data set before feature engineering LengthOfStay.

**Output:** Data set with new features LoS.

In [8]:
# Get the mean and standard deviation of continuous variables.
col_list = rx_get_var_names(LengthOfStay_cleaned_sql)
f = "+".join(col_list)
summary = rx_summary(formula = f, data = LengthOfStay_cleaned_sql, by_term = True).summary_data_frame

names = ["hematocrit", "neutrophils", "sodium", "glucose", "bloodureanitro", "creatinine", "bmi", "pulse", "respiration"]
statistics = summary[summary["Name"].isin(names)]
statistics = statistics[["Name", "Mean", "StdDev"]]

# standardization transform function
def standardize(data, context):
    for n, row in statistics.iterrows():
        data[[row["Name"]]] = (data[[row["Name"]]] - row["Mean"])/row["StdDev"]
    return data

# number_of_issues transform function
def calculate_number_of_issues(data, context):
    data["number_of_issues"] = to_numeric(data["hemo"]) + to_numeric(data["dialysisrenalendstage"]) + to_numeric(data["asthma"])\
                               + to_numeric(data["irondef"]) + to_numeric(data["pneum"]) + to_numeric(data["substancedependence"])\
                               + to_numeric(data["psychologicaldisordermajor"]) + to_numeric(data["depress"])\
                               + to_numeric(data["psychother"]) + to_numeric(data["fibrosisandother"]) + to_numeric(data["malnutrition"])
    return data

# Combine transform functions into one overarching transform
def transform(dataset, context):
    data = DataFrame(dataset)
    data = standardize(data, context)
    data = calculate_number_of_issues(data, context)
    return data

# We drop the LoS view in case the SQL Stored Procedure was executed in the same database before.
drop_view("LoS", connection_string)

# Standardize the cleaned table by wrapping it up in rxDataStep. Output is written to LoS_standard.
table_name = "LengthOfStay" if missing is False else "LoS0"
LengthOfStay_cleaned_sql = RxSqlServerData(sql_query = "SELECT * FROM [Hospital].[dbo].[{}]".format(table_name),
                                           connection_string = connection_string)
LoS_sql = RxSqlServerData(table = "LoS", connection_string = connection_string)
rx_data_step(input_data = LengthOfStay_cleaned_sql, output_file = LoS_sql, overwrite = True, transform_function = transform)

# Converting number_of_issues to character with a SQL query because as.character in rxDataStep is crashing.
alter_column("LoS", "number_of_issues", "varchar(2)", connection_string)
alter_column("LoS", "lengthofstay", "float", connection_string)

print("Feature Engineering Completed")

Rows Read: 50000, Total Rows Processed: 50000, Total Chunk Time: 1.594 seconds
Rows Read: 50000, Total Rows Processed: 100000, Total Chunk Time: 1.584 seconds 
Computation time: 3.338 seconds.
Rows Read: 100000, Total Rows Processed: 100000Total Rows written: 100000, Total time: 5.37
, Total Chunk Time: 10.949 seconds 
Feature Engineering Completed


## Step 3: Training and Evaluating the Models

In this step we:

**1.** Split randomly the data set LoS into a training (LoS_Train) and a testing (LoS_Test) set.
 
**2.** Train a Random Forest, Boosted Trees, Fast Trees, and Neural Network models on LoS_Train, and save them to SQL. 

**3.** Score the models on LoS_Test.

**Input:** Data set LoS.

**Output:** Random forest, Boosted Trees, Fast Trees, and Neural Network models saved to SQL and performance metrics.  

In [9]:
# Point to the SQL table with the data set for modeling. Strings will be converted to factors.
LoS = RxSqlServerData(table = "LoS", connection_string = connection_string, strings_as_factors = True)

print(col_type_and_factor_info)

{'bloodureanitro': {'type': 'numeric'}, 'vdate': {'type': 'factor'}, 'glucose': {'type': 'numeric'}, 'irondef': {'type': 'factor', 'levels': ['0', '1']}, 'hematocrit': {'type': 'numeric'}, 'dialysisrenalendstage': {'type': 'factor', 'levels': ['0', '1']}, 'discharged': {'type': 'factor'}, 'neutrophils': {'type': 'numeric'}, 'bmi': {'type': 'numeric'}, 'hemo': {'type': 'factor', 'levels': ['0', '1']}, 'creatinine': {'type': 'numeric'}, 'number_of_issues': {'type': 'factor', 'levels': ['0', '2', '1', '3', '4', '5', '6', '7', '8', '9']}, 'malnutrition': {'type': 'factor', 'levels': ['0', '1']}, 'eid': {'type': 'integer'}, 'lengthofstay': {'type': 'numeric'}, 'psychologicaldisordermajor': {'type': 'factor', 'levels': ['0', '1']}, 'pneum': {'type': 'factor', 'levels': ['0', '1']}, 'secondarydiagnosisnonicd9': {'type': 'factor', 'levels': ['4', '1', '2', '3', '0', '7', '6', '10', '8', '5', '9']}, 'fibrosisandother': {'type': 'factor', 'levels': ['0', '1']}, 'rcount': {'type': 'factor', 'leve

In [10]:
# Randomly split the data into a training set and a testing set, with a splitting % p.
# p % goes to the training set, and the rest goes to the testing set. Default is 70%.

p = 70

## Create the Train_Id table containing Lead_Id of training set.
train_test_split("eid", "LoS", "Train_Id", p, connection_string)

## Point to the training set. It will be created on the fly when training models.
variables_all = rx_get_var_names(LoS)
variables_to_remove = ["eid", "vdate", "discharged", "facid"]
training_variables = [x for x in variables_all if x not in variables_to_remove]
LoS_Train = RxSqlServerData(sql_query = "SELECT eid, {} FROM LoS WHERE eid IN (SELECT eid from Train_Id)".format(
    ', '.join(training_variables)), connection_string = connection_string, column_info = col_type_and_factor_info
)

## Point to the testing set. It will be created on the fly when testing models.
LoS_Test = RxSqlServerData(sql_query = "SELECT eid, {} FROM LoS WHERE eid NOT IN (SELECT eid from Train_Id)".format(
    ', '.join(training_variables)), connection_string = connection_string, column_info = col_type_and_factor_info
)

print("Splitting completed")

Splitting completed


In [11]:
# Write the formula after removing variables not used in the modeling.
formula = create_formula("lengthofstay", variables_all, variables_to_remove)
print("Formula: ", formula)

Formula:  lengthofstay ~ rcount + gender + dialysisrenalendstage + asthma + irondef + pneum + substancedependence + psychologicaldisordermajor + depress + psychother + fibrosisandother + malnutrition + hemo + hematocrit + neutrophils + sodium + glucose + bloodureanitro + creatinine + bmi + pulse + respiration + secondarydiagnosisnonicd9 + number_of_issues


In [12]:
num_rows = get_num_rows("Train_Id", connection_string)

# Define functions to tune rx_dforest and rx_btrees
def tune_rx_dforest(formula, data, n_tree_list, cp_list, cc):
    print("Tuning rx_dforest")
    best_error = sys.maxsize
    best_model = None
    for nt in n_tree_list:
        for cp in cp_list:
            model = rx_dforest(formula=formula,
                               data=data,
                               n_tree=nt,
                               cp=cp,
                               min_split=int(sqrt(num_rows)),
                               max_num_bins=int(sqrt(num_rows)),
                               seed=5,
                               compute_context=cc)
            error = model.oob_err['oob.err'][model.ntree - 1]
            print("OOB Error: {} \t n_tree: {} \t cp: {}".format(error, nt, cp))
            if error < best_error:
                best_error = error
                best_model = model
    return best_model


def tune_rx_btrees(formula, data, n_tree_list, lr_list, cp_list, cc):
    print("Tuning rx_btrees")
    best_error = sys.maxsize
    best_model = None
    for nt in n_tree_list:
        for lr in lr_list:
            for cp in cp_list:
                model = rx_btrees(formula=formula,
                                  data=data,
                                  n_tree=nt,
                                  learning_rate=lr,
                                  cp=cp,
                                  loss_function="gaussian",
                                  min_split=int(sqrt(num_rows)),
                                  max_num_bins=int(sqrt(num_rows)),
                                  seed=9,
                                  compute_context=cc)
                error = model.oob_err['oob.err'][model.ntree - 1]
                print("OOB Error: {} \t n_tree: {} \t learning_rate: {} \t cp: {}".format(error, nt, lr, cp))
                if error < best_error:
                    print("^^^ New best model!")
                    best_error = error
                    best_model = model
    return best_model

Rows Read: 1, Total Rows Processed: 1, Total Chunk Time: 0.006 seconds 


In [13]:
# Train the Random Forest.
forest_model = tune_rx_dforest(formula, LoS_Train, n_tree_list=[40], cp_list=[0.00005], cc=sql)

print("Training Regression RF done")

Tuning rx_dforest
Number of observations not available for this data source. 'numObs' set to 1e6.

For this data source, explicitly set 'max_num_bins' and 'min_split', otherwise innacurate models can result.
Please see rx_dtree 'max_num_bins' and 'min_split' for details.
OOB Error: 0.7229219675064087 	 n_tree: 40 	 cp: 5e-05
Training Regression RF done


In [14]:
# Save the Random Forest in SQL. The compute context is set to local in order to export the model.
write_rts_model(forest_model, "RF", connection_string)

print("RF model uploaded to SQL")

Rows Read: 1, Total Rows Processed: 1
Total Rows written: 1, Total time: 0.011
, Total Chunk Time: 0.050 seconds 
RF model uploaded to SQL


In [15]:
# Train the Boosted Trees model. This tunes on the basis of minimizing oob error.
boosted_model = tune_rx_btrees(formula, LoS_Train, n_tree_list=[40], lr_list=[0.3], cp_list=[0.00005], cc=sql)

Tuning rx_btrees
Number of observations not available for this data source. 'numObs' set to 1e6.

For this data source, explicitly set 'max_num_bins' and 'min_split', otherwise innacurate models can result.
Please see rx_dtree 'max_num_bins' and 'min_split' for details.
OOB Error: 0.7843199372291565 	 n_tree: 40 	 learning_rate: 0.3 	 cp: 5e-05
^^^ New best model!


In [16]:
# Save the Boosted Trees in SQL. The compute context is set to Local in order to export the model.
rx_set_compute_context(local)

write_rts_model(boosted_model, "GBT", connection_string)

Rows Read: 1, Total Rows Processed: 1
Total Rows written: 1, Total time: 0.001
, Total Chunk Time: 0.033 seconds 


In [17]:
# Train the Fast Trees model.
fast_model = rx_fast_trees(formula=formula,
                          data=LoS_Train,
                          method="regression",
                          num_trees=40,
                          learning_rate=0.2,
                          split_fraction=5/24,
                          min_split=10,
                          compute_context=sql)

print("Training Fast Trees done")

'unbalanced_sets' ignored for method 'regression'
Elapsed time: 00:00:00.8574579
Training Fast Trees done


In [18]:
# Save the Fast Trees in SQL. The compute context is set to Local in order to export the model.
write_rts_model(fast_model, "FT", connection_string)

print("Fast Trees model uploaded to SQL")

Rows Read: 1, Total Rows Processed: 1
Total Rows written: 1, Total time: 0.002
, Total Chunk Time: 0.033 seconds 
Fast Trees model uploaded to SQL


In [19]:
# Train the Neural Network model.
NN_model = rx_neural_network(formula=formula,
                            data=LoS_Train,
                            method="regression",
                            num_hidden_nodes=128,
                            num_iterations=100,
                            optimizer=adadelta_optimizer(),
                            mini_batch_size=20,
                            compute_context=sql)

print("Training Neural Network done")

Elapsed time: 00:00:00.1026195
Training Neural Network done


In [20]:
write_rts_model(NN_model, "NN", connection_string)

Rows Read: 1, Total Rows Processed: 1
Total Rows written: 1, Total time: 0.001
, Total Chunk Time: 0.034 seconds 


In [21]:
# Random Forest Scoring 

# Make Predictions, then import them into Python.
forest_prediction_sql = RxSqlServerData(table = "Forest_Prediction",
                                        strings_as_factors = True,
                                        connection_string = connection_string)
rx_predict(forest_model,
           data = LoS_Test,
           output_data = forest_prediction_sql,
           type = "response",
           extra_vars_to_write = ["lengthofstay", "eid"],
           overwrite = True)

# Compute the performance metrics of the model.
forest_prediction = rx_import(input_data = forest_prediction_sql)
forest_metrics = evaluate_model(observed = forest_prediction['lengthofstay'],
                                predicted = forest_prediction['lengthofstay_Pred'],
                                model = "RF")

print("Scoring Random Forest (rxDForest) done")

Rows Read: 29938, Total Rows Processed: 29938, Total Chunk Time: 1.321 seconds
Total Rows written: 29938, Total time: 0.499
 
Rows Read: 29938, Total Rows Processed: 29938, Total Chunk Time: 0.031 seconds 
OrderedDict([('model_name', ['RF']), ('mean_absolute_error', [0.6188585992512238]), ('root_mean_squared_error', [0.84188309193952404]), ('relative_absolute_error', [0.32274628245733628]), ('relative_squared_error', [0.12710825557031963]), ('coefficient_of_determination', [0.87289174442968043])])
Scoring Random Forest (rxDForest) done


In [22]:
# Boosted Trees Scoring
## Make Predictions, then import them into R. 
boosted_prediction_sql = RxSqlServerData(table = "Boosted_Prediction",
                                         strings_as_factors = True,
                                         connection_string = connection_string)
rx_predict(boosted_model,
           data = LoS_Test,
           output_data = boosted_prediction_sql,
           extra_vars_to_write = ["lengthofstay", "eid"],
           overwrite = True)

# Compute the performance metrics of the model.
boosted_prediction = rx_import(input_data = boosted_prediction_sql)
boosted_metrics = evaluate_model(observed = boosted_prediction['lengthofstay'],
                                 predicted = boosted_prediction['lengthofstay_Pred'],
                                 model = "GBT")

print("Scoring Boosted Trees (rx_btrees) done")

Rows Read: 29938, Total Rows Processed: 29938, Total Chunk Time: 1.286 seconds
Total Rows written: 29938, Total time: 0.507
 
Rows Read: 29938, Total Rows Processed: 29938, Total Chunk Time: 0.031 seconds 
OrderedDict([('model_name', ['GBT']), ('mean_absolute_error', [0.6511701891108345]), ('root_mean_squared_error', [0.88104416506831673]), ('relative_absolute_error', [0.33959737820052077]), ('relative_squared_error', [0.13920843219290197]), ('coefficient_of_determination', [0.86079156780709809])])
Scoring Boosted Trees (rx_btrees) done


In [23]:
# Make Predictions, then write them to a table.
LoS_Test_import = rx_import(input_data=LoS_Test)
fast_prediction = ml_predict(fast_model, data=LoS_Test_import, extra_vars_to_write=["lengthofstay", "eid"], overwrite=True)
fast_prediction_sql = RxSqlServerData(table="Fast_Prediction", strings_as_factors=True, connection_string=connection_string)
rx_data_step(input_data=fast_prediction, output_file=fast_prediction_sql, overwrite=True)

# Compute the performance metrics of the model.
fast_metrics = evaluate_model(observed=fast_prediction['lengthofstay'], predicted=fast_prediction['Score'], model="FT")

print("Scoring Fast Trees (rx_fast_trees) done")

Rows Read: 29938, Total Rows Processed: 29938, Total Chunk Time: 1.205 seconds 
Beginning processing data.
Rows Read: 29938, Read Time: 0, Transform Time: 0
Beginning processing data.
Elapsed time: 00:00:00.5841457
Finished writing 29938 rows.
Writing completed.
Rows Read: 29938, Total Rows Processed: 29938
Total Rows written: 29938, Total time: 0.491
, Total Chunk Time: 0.536 seconds 
OrderedDict([('model_name', ['FT']), ('mean_absolute_error', [0.3701010029037536]), ('root_mean_squared_error', [0.52396516787604697]), ('relative_absolute_error', [0.19301456417579552]), ('relative_squared_error', [0.049235121909362738]), ('coefficient_of_determination', [0.95076487809063726])])
Scoring Fast Trees (rx_fast_trees) done


In [24]:
# Make Predictions, then write them to a table.
NN_prediction = ml_predict(NN_model, data=LoS_Test_import, extra_vars_to_write=["lengthofstay", "eid"], overwrite=True)
NN_prediction_sql = RxSqlServerData(table="NN_Prediction", strings_as_factors=True, connection_string=connection_string)
rx_data_step(input_data=NN_prediction, output_file=NN_prediction_sql, overwrite=True)

# Compute the performance metrics of the model.
NN_metrics = evaluate_model(observed=NN_prediction['lengthofstay'], predicted=NN_prediction['Score'], model="NN")

print("Scoring Neural Networks (rx_neural_networks) done")

Beginning processing data.
Rows Read: 29938, Read Time: 0.001, Transform Time: 0
Beginning processing data.
Elapsed time: 00:00:00.3213019
Finished writing 29938 rows.
Writing completed.
Rows Read: 29938, Total Rows Processed: 29938
Total Rows written: 29938, Total time: 0.486
, Total Chunk Time: 0.602 seconds 
OrderedDict([('model_name', ['NN']), ('mean_absolute_error', [0.4887321793616547]), ('root_mean_squared_error', [0.70966139753137347]), ('relative_absolute_error', [0.25488293157289299]), ('relative_squared_error', [0.090317633153016302]), ('coefficient_of_determination', [0.90968236684698367])])
Scoring Neural Networks (rx_neural_networks) done
